<a href="https://colab.research.google.com/github/prottushee/AI_Project-FakeNewsDetector-/blob/main/FakeNewsDetector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Concatenate

In [ ]:
real_news_data = pd.read_csv('/content/drive/MyDrive/True.csv')
fake_news_data = pd.read_csv('/content/drive/MyDrive/Fake.csv')

In [ ]:
# Add a new column 'is_real' to indicate real or fake news
real_news_data['is_real'] = 1
fake_news_data['is_real'] = 0


In [ ]:
data = pd.concat([real_news_data, fake_news_data],ignore_index=True )



In [ ]:
text_columns = ['title', 'text', 'date']
combined_text = data[text_columns].apply(lambda row: ' '.join(row.values.astype(str)), axis=1)


In [ ]:
# Tokenize and pad the text data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(combined_text)
sequences = tokenizer.texts_to_sequences(combined_text)
padded_sequences = pad_sequences(sequences, maxlen=100, padding='post', truncating='post')


In [ ]:
# Prepare the target variable
targets = data['is_real'].astype(int)  # Assuming the target column is named 'is_real' (0 for fake, 1 for real)


In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(padded_sequences, targets, test_size=0.2, random_state=42)
# y_train

In [ ]:
vocab_size = len(tokenizer.word_index) + 1
embedding_dim = 100
max_length = 100

model = Sequential([
    Embedding(vocab_size, embedding_dim, input_length=max_length),
    LSTM(64, return_sequences=True),
    LSTM(64),
    Dropout(0.2),
    Dense(64, activation='relu'),
    Dropout(0.2),
    Dense(1, activation='sigmoid')
])


In [ ]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])



In [ ]:
model.fit(X_train, y_train, epochs=5, batch_size=32, validation_data=(X_test, y_test))


Epoch 1/5
1123/1123 [==============================] - 63s 50ms/step - loss: 0.2675 - accuracy: 0.8934 - val_loss: 0.1105 - val_accuracy: 0.9763
Epoch 2/5
1123/1123 [==============================] - 22s 20ms/step - loss: 0.0834 - accuracy: 0.9808 - val_loss: 0.3839 - val_accuracy: 0.8415
Epoch 3/5
1123/1123 [==============================] - 21s 19ms/step - loss: 0.1348 - accuracy: 0.9669 - val_loss: 0.1570 - val_accuracy: 0.9639
Epoch 4/5
1123/1123 [==============================] - 20s 18ms/step - loss: 0.1196 - accuracy: 0.9616 - val_loss: 0.0593 - val_accuracy: 0.9826
Epoch 5/5
1123/1123 [==============================] - 22s 19ms/step - loss: 0.0192 - accuracy: 0.9953 - val_loss: 0.0245 - val_accuracy: 0.9944


In [ ]:
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f'Test Accuracy: {test_acc}')

281/281 [==============================] - 2s 6ms/step - loss: 0.0245 - accuracy: 0.9944
Test Accuracy: 0.9944320917129517
